## Pydantic v2

In [ ]:
import pydantic

## Without using pydantic

In [ ]:
class User:
    def __init__(self, id: int, name: str="Alice"):
        if not isinstance(id, int):
            raise TypeError(f"Expected id to be an integer but got {type(id).__name__}")
        
        self.id = id
        self.name = name
        
    def __repr__(self):
        return f"User(id={self.id}, name={self.name})"

In [9]:
try:
    user = User(id="1")
except TypeError as e:
    print(e)

Expected id to be an integer but got str


In [10]:
try:
    user = User(id=1)
    print(user)
except TypeError as e:
    print(e)

User(id=1, name=Alice)


## Using Pydantic

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = "Alice"

In [13]:
user = User(id=1)
user

User(id=1, name='Alice')

In [16]:
try:
    user = User(id="a")
    print(user)
except:
    print("Error")

Error


In [18]:
print(user.model_fields_set)  # not include default value

{'id'}


In [21]:
user = User(id=1, name="Bob")
print(user.model_fields_set)

{'name', 'id'}


In [22]:
print(user.model_dump())
print(user.model_dump_json())
print(user.model_json_schema())

{'id': 1, 'name': 'Bob'}
{"id":1,"name":"Bob"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'Alice', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


## Nested

In [24]:
from typing import List, Optional
from pydantic import BaseModel

class Food(BaseModel):
    name: str
    price: float
    ingredients: Optional[List[str]] = None
    
class Menu(BaseModel):
    foods: List[Food]

menu_instance = Menu(
    foods=[
        {"name": "apple", "price": 100.0},
        {"name": "banana", "price": 200.0,"ingredients": ["a", "b", "e"]}
    ]
)

print(menu_instance)
print(menu_instance.model_dump())

foods=[Food(name='apple', price=100.0, ingredients=None), Food(name='banana', price=200.0, ingredients=['a', 'b', 'e'])]
{'foods': [{'name': 'apple', 'price': 100.0, 'ingredients': None}, {'name': 'banana', 'price': 200.0, 'ingredients': ['a', 'b', 'e']}]}


In [25]:
# !pip install pydantic[email]

In [29]:
from typing import List
from pydantic import BaseModel, EmailStr, conlist, Field, HttpUrl, PositiveInt

class Address(BaseModel):
    location: str
    
class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr
    
class Owner(BaseModel):
    name: str
    email: EmailStr
    contact: Optional[str] = None

class Restaurant(BaseModel):
    name: str = Field(..., pattern=r"^[A-Z].*") # required start with capital letter
    owner: Owner
    address: Address
    # employees: conlist(Employee, min_items=2)
    number_of_seats: PositiveInt
    employees: List[Employee]
    website: HttpUrl
    
    @pydantic.field_validator("employees")
    def validate_employees(cls, v):
        if len(v) < 2:
            raise ValueError("There should be at least 2 employees")
        return v

In [32]:
restuarant_instance = Restaurant(
    name = "Xyz", # validation here should be Capitalized
    owner = {
        "name": "Alice",
        "email": "alice@xyz.com", # email validation here
    },
    address = {
        "location": "123"
    },
    number_of_seats=10,
    employees=[ # min number of employee should be 2
        {
            "name": "Bob",
            "position": "chef",
            "email": "bob@xyz.com"
        },
        {
            "name": "Bob2",
            "position": "chef",
            "email": "bob@xyz.com"
        }
    ],
    website = "http://xyz.com" # website validation here
)

In [34]:
print(restuarant_instance)

name='Xyz' owner=Owner(name='Alice', email='alice@xyz.com', contact=None) address=Address(location='123') number_of_seats=10 employees=[Employee(name='Bob', position='chef', email='bob@xyz.com'), Employee(name='Bob2', position='chef', email='bob@xyz.com')] website=Url('http://xyz.com/')


In [35]:
print(restuarant_instance.model_dump())

{'name': 'Xyz', 'owner': {'name': 'Alice', 'email': 'alice@xyz.com', 'contact': None}, 'address': {'location': '123'}, 'number_of_seats': 10, 'employees': [{'name': 'Bob', 'position': 'chef', 'email': 'bob@xyz.com'}, {'name': 'Bob2', 'position': 'chef', 'email': 'bob@xyz.com'}], 'website': Url('http://xyz.com/')}


In [36]:
print(restuarant_instance.model_dump_json())

{"name":"Xyz","owner":{"name":"Alice","email":"alice@xyz.com","contact":null},"address":{"location":"123"},"number_of_seats":10,"employees":[{"name":"Bob","position":"chef","email":"bob@xyz.com"},{"name":"Bob2","position":"chef","email":"bob@xyz.com"}],"website":"http://xyz.com/"}
